# sPyTial Dataclass Builder with CnD-core

This notebook demonstrates the new CnD-core based dataclass builder widget that provides **visual, interactive construction** of dataclass instances.

## Key Features

- **Visual Construction**: Uses CnD-core's `structured-input-graph` for interactive building
- **No File I/O**: Direct JavaScript-Python communication via postMessage
- **Real-time Value**: `widget.value` always contains the current dataclass instance
- **Spatial Annotations**: Automatically includes any spatial constraints from decorators

## Installation

Make sure you have ipywidgets installed:

```bash
pip install ipywidgets
jupyter nbextension enable --py widgetsnbextension
```

## Basic Usage

In [1]:
from dataclasses import dataclass
import spytial

In [2]:
@dataclass
class Person:
    name: str = "placeholder"
    age: int = 0
    email: str = "p2"

### Create the Builder Widget

This creates an interactive CnD-core based visual builder:

In [3]:
person_widget = spytial.dataclass_builder(Person)
person_widget

### Instructions:

1. Use the visual interface above to build your `Person` instance
2. Click the **"Export JSON"** button when done
3. The data is automatically captured in `person_widget.value`

In [ ]:
# Access the built instance

## Closer! I think we still need to make some changes -- stale - older values keep being read. We need to 


person = person_widget.value
print(f"Built person: {person}")
print(f"Name: {person.name if person else 'Not built yet'}")

Built person: Person(name='Alice', age=30, email='test@example.com')
Name: Alice


## Debugging Export

If export isn't working, run this cell to check the console logs:

In [14]:
# Check if widget is registered and accessible
print(f"Widget ID: {person_widget._widget_id}")
print(f"Widget type: {type(person_widget).__name__}")
print(f"Current value: {person_widget._current_value}")

# Check global registry
import sys
if 'spytial.dataclass_widget_cnd' in sys.modules:
    widget_module = sys.modules['spytial.dataclass_widget_cnd']
    if hasattr(widget_module, '_spytial_widgets'):
        print(f"\n✓ Global registry exists in module")
        print(f"  Registered widgets: {list(widget_module._spytial_widgets.keys())}")
    else:
        print("\n✗ Global registry NOT in module")
else:
    print("\n✗ Module not loaded")

# Check if accessible from globals()
if '_spytial_widgets' in globals():
    print(f"\n✓ Registry accessible from globals()")
    print(f"  Widgets: {list(globals()['_spytial_widgets'].keys())}")
else:
    print("\n✗ Registry NOT in globals() - This is the problem!")
    print("  The kernel execution can't find the widget registry")


Widget ID: spytial_4368388784
Widget type: DataclassBuilderWidget
Current value: Person(name='Alice', age=30, email='test@example.com')

✓ Global registry exists in module
  Registered widgets: ['spytial_4368388784', 'spytial_4368395456', 'spytial_4368398240']

✓ Registry accessible from globals()
  Widgets: ['spytial_4368388784', 'spytial_4368395456', 'spytial_4368398240']


In [6]:
# Manual test: Simulate what happens when Export is clicked
# This bypasses the JavaScript and tests the Python side directly
from spytial.dataclass_widget_cnd import _spytial_widgets, _json_to_dataclass

print("Testing manual data update...")
test_data = {
    'atoms': [
        {'id': 'n0', 'label': 'Person', 'type': ['Person']},
        {'id': 'n1', 'label': 'Alice', 'value': 'Alice'},
        {'id': 'n2', 'label': '30', 'value': 30},
        {'id': 'n3', 'label': 'test@example.com', 'value': 'test@example.com'}
    ],
    'relations': [
        {'id': 'r0', 'source': 'n0', 'target': 'n1', 'label': 'name'},
        {'id': 'r1', 'source': 'n0', 'target': 'n2', 'label': 'age'},
        {'id': 'r2', 'source': 'n0', 'target': 'n3', 'label': 'email'}
    ]
}

# Update the widget directly
person_widget._current_value = _json_to_dataclass(test_data, Person)
print(f"✅ Manually set value: {person_widget.value}")
print("\nIf this works but clicking Export doesn't, the issue is in JavaScript → Python communication")

Testing manual data update...
✅ Manually set value: Person(name='Alice', age=30, email='test@example.com')

If this works but clicking Export doesn't, the issue is in JavaScript → Python communication


## With Spatial Annotations

The widget automatically includes spatial constraints from sPyTial decorators:

In [7]:
@dataclass
@spytial.orientation(selector='self', directions=['horizontal'])
class TreeNode:
    value: int = 0
    left: 'TreeNode' = None
    right: 'TreeNode' = None

In [8]:
tree_widget = spytial.dataclass_builder(TreeNode)
tree_widget

In [9]:
# The widget respects spatial annotations from decorators
tree = tree_widget.value
print(f"Built tree: {tree}")

Built tree: None


## Workflow: Build → Visualize

A common workflow is to build data with the widget, then visualize it:

In [10]:
# Step 1: Build your data
widget = spytial.dataclass_builder(Person)
widget

In [11]:
# Step 2: After building and exporting, visualize it
if widget.value:
    spytial.diagram(widget.value)

## Architecture

### How It Works

1. **CnD Spec Generation**: Extracts spatial annotations from decorators and generates YAML spec
2. **Initial Data**: Creates empty dataclass instance and converts to CnD data format
3. **Visual Interface**: Embeds `structured-input-graph` web component in iframe
4. **Export Handler**: JavaScript postMessage sends data back to Python kernel
5. **Value Update**: Automatically converts JSON to dataclass and updates `widget.value`

### No File I/O, No Timeouts

Unlike the old system:
- ❌ No temporary files or export directories
- ❌ No polling or timeout mechanisms
- ✅ Direct kernel communication via `window.Jupyter.notebook.kernel.execute()`
- ✅ Immediate value availability at `widget.value`

### CnD-core Integration

Uses the following CnD components:
- `structured-input-graph`: Visual construction interface
- `JSONDataInstance`: Data instance format
- CnD spec (YAML): Spatial constraint specification

## Comparison: Old vs New

### Old System (dataclass_widget.py)
```python
# Complex file-based system
widget = spytial.dataclass_widget(Person)
# Wait for file export...
# Poll directory...
# Hope timeout doesn't expire...
person = widget.value  # Might be None
```

### New System (dataclass_widget_cnd.py)
```python
# Simple, direct communication
widget = spytial.dataclass_builder(Person)
# Build visually, click Export
person = widget.value  # Immediately available
```